In [ ]:
conda create -n ai python=3.11 -y && conda activate ai

In [ ]:
pip install torch
pip install git+https://github.com/huggingface/transformers
pip install git+https://github.com/huggingface/accelerate
pip install huggingface_hub

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
login(token="Your Token")
tokenizer = AutoTokenizer.from_pretrained("fdtn-ai/Foundation-Sec-8B")
model = AutoModelForCausalLM.from_pretrained("fdtn-ai/Foundation-Sec-8B")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
alert = {
    "src_ip": "192.168.10.12",
    "dst_ip": "10.0.0.5",
    "src_port": 554,
    "dst_port": 443,
    "protocol": "TCP",
    "payload": "GET /admin/config.php?cmd=rm -rf / HTTP/1.1"
}

# Prompt for MITRE ATT&CK mapping
prompt = f"""
You are a cybersecurity analyst. Given the following network alert, map it to MITRE ATT&CK techniques.

Alert:
- Source IP: {alert['src_ip']}
- Destination IP: {alert['dst_ip']}
- Source Port: {alert['src_port']}
- Destination Port: {alert['dst_port']}
- Protocol: {alert['protocol']}
- Payload: {alert['payload']}

Which MITRE ATT&CK techniques does this alert match? Return the technique name and ID.
"""

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    add_special_tokens=True
)

inputs = {k: v.to(device) for k, v in inputs.items()}
pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=550,
    do_sample=True,
    temperature=0.1,
    top_p=0.9,
    pad_token_id=pad_token_id
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.replace(prompt, "").strip()
print(response)

In [ ]:
attack_scenario = {
    "target": "e-commerce web server running WordPress",
    "entry_point": "exposed admin interface",
    "constraints": "avoid detection, maintain persistence"
}

# Construct prompt
prompt = f"""Simulate a realistic cyber attack sequence based on these parameters:
Target: {attack_scenario['target']}
Initial Access: {attack_scenario['entry_point']}
Constraints: {attack_scenario['constraints']}

Generate attack steps in MITRE ATT&CK order:
1. Reconnaissance:"""

# Generate simulation
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=400,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response.split("Generate attack steps")[0])  # Remove original promp

In [ ]:
# Construct prompt
prompt = f"""Simulate lateral movement using Pass-the-Hash after obtaining credentials from a Windows workstation. Include steps to exploit SMB vulnerabilities."""

# Generate simulation
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=400,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response.split("Generate attack steps")[0])  # Remove original prompt
print(response)